In this notebook, we'll demonstrate how to retrieve data automatically from draft guru

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from collections import ChainMap
import math
import numpy as np

In [2]:
def make_soup(website):
    if 'https' in website:
        page = requests.get(website, timeout=120)
    else:
        page = requests.get('https://' + website, timeout=120)
    soup = BeautifulSoup(page.content, 'html.parser')
    page.close()
    return soup

In [3]:
TEAMS[0]

NameError: name 'TEAMS' is not defined

In [4]:
URL = "https://www.footywire.com/afl/footy/brownlow_medal?year=2022&s=V"
soup = make_soup(URL)
soup


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<title>Brownlow Medal</title>
<meta content="AFL Brownlow Medal history and statistics." name="description"/>
<meta content="AFL Brownlow Medal" name="keywords"/>
<script>
var ad2Div;
var ad2Stuck = false;

var ad2Reversed;

var headerDiv;
var headerHeight;

var contentPageCell;
var contentPageRect;
var ad2Rect;
var ad2BottomPx;

var scrollOffset = window.scrollY;
var lastScrollOffset = window.scrollY;
var lastScrollIncrement = 0;
var scrolledDown = false;
var scrollReversed = false;

var contentPageCellHeightChecked = false;

function setAd2Properties() {
  ad2Div = document.getElementById("scrollad");
  if (ad2Div) {
    ad2Stuck = false;
    ad2Reversed = false;
  }
}

function setHeaderProperties() {
  headerDiv = document.getElementById("header");
  if (headerDiv) {
    headerHeight = headerDiv.offsetHeight;
  }
}
function setContentCellProperties() {
  contentPageCell = document.getElementById('conten

In [5]:


#Actually better to search for "td = content page cell" - rather than searching for the width
brownlow_table_html = [a for a in soup.find_all('table') if a.get('width')=='688'][0]
brownlow_table_html

<table border="0" cellpadding="0" cellspacing="0" width="688">
<tr>
<td class="lbnorm" height="28"> Player</td>
<td class="lbnorm" width="110">Team</td>
<td class="bnorm" width="50"><span class="sortByAjaxLink" onclick="getSbyValue('V')" title="Total Votes">V</span></td>
<td class="bnorm" width="50"><span class="sortByAjaxLink" onclick="getSbyValue('3')" title="3-Vote Games">3V</span></td>
<td class="bnorm" width="50"><span class="sortByAjaxLink" onclick="getSbyValue('2')" title="2-Vote Games">2V</span></td>
<td class="bnorm" width="50"><span class="sortByAjaxLink" onclick="getSbyValue('1')" title="1-Vote Games">1V</span></td>
<td class="bnorm" width="60"><span class="sortByAjaxLink" onclick="getSbyValue('G')" title="Games Played">Played</span></td>
<td class="bnorm" width="60"><span class="sortByAjaxLink" onclick="getSbyValue('P')" title="Games Polled">Polled</span></td>
<td class="bnorm" width="55"><span class="sortByAjaxLink" onclick="getSbyValue('A')" title="Votes Per Game">V/G</sp

In [8]:
def clean_text(a):
    return a.replace('\xa0','').rstrip().lstrip()

def get_ids(row_data):
    ids = [a.find('a')['href'] for a in row_data if a.find('a') is not None]
    player_id = [x.replace("pp-","") for x in ids if "pp-" in x][0]
    team_id = [x.replace("tb-","") for x in ids if "tb-" in x][0].split('?')[0]
    return player_id, team_id 
    
    
def parse_row_data(row_data,headers):
    data_dict = {headers[i]:clean_text(row_data[i].text) for i in range(len(row_data))}
    pid,tid = get_ids(row_data)
    data_dict['player_id'] = pid
    data_dict['team_id'] =  tid
    return data_dict
    
flat_table_data = [a for a in brownlow_table_html.find_all('td')]
header_elements = flat_table_data[0:9]
body_elements = flat_table_data[9:]
headers = [clean_text(a.text) for a in header_elements]
data_list = []
for e in range(0,len(body),len(header)):
    row_data = body_elements[e:e+len(header)]
    data_list.append(parse_row_data(row_data,headers))
pd.DataFrame(data_list)

,Player,Team,V,3V,2V,1V,Played,Polled,V/G,player_id,team_id
0,Patrick Cripps W,Blues,29,7,2,4,21,13,1.38,carlton-blues--patrick-cripps,carlton-blues
1,Lachie Neale,Lions,28,7,3,1,22,11,1.27,brisbane-lions--lachie-neale,brisbane-lions
2,Touk Miller,Suns,27,4,6,3,22,13,1.23,gold-coast-suns--touk-miller,gold-coast-suns
3,Andrew Brayshaw,Dockers,25,5,4,2,22,11,1.14,fremantle-dockers--andrew-brayshaw,fremantle-dockers
4,Clayton Oliver,Demons,25,4,5,3,21,12,1.19,melbourne-demons--clayton-oliver,melbourne-demons
...,...,...,...,...,...,...,...,...,...,...,...
202,Tim Membrey,Saints,1,0,0,1,22,1,0.05,st-kilda-saints--tim-membrey,st-kilda-saints
203,Sebastian Ross,Saints,1,0,0,1,21,1,0.05,st-kilda-saints--sebastian-ross,st-kilda-saints
204,Jake Lloyd,Swans,1,0,0,1,21,1,0.05,sydney-swans--jake-lloyd,sydney-swans
205,Willie Rioli,Eagles,1,0,0,1,13,1,0.08,west-coast-eagles--willie-rioli,west-coast-eagles


In [9]:
def clean_text(a):
    return a.replace('\xa0','').rstrip().lstrip()

def get_pids(row_data):
    ids = [a.find('a')['href'] for a in row_data if a.find('a') is not None]
    player_id = [x.replace("pp-","") for x in ids if "pp-" in x][0]
    return clean_text(player_id) 
    
def get_tids(row_data):
    ids = [a.find('a')['href'] for a in row_data if a.find('a') is not None]
    team_id = [x.replace("tb-","") for x in ids if "tb-" in x][0].split('?')[0]
    return clean_text(team_id) 
       
def parse_row_data_bl(row_data,headers):
    data_dict = {headers[i]:clean_text(row_data[i].text) for i in range(len(row_data))}
    pid = get_pids(row_data)
    tid = get_tids(row_data)
    data_dict['player_id'] = pid
    data_dict['team_id'] =  tid
    return data_dict
    

def get_brownlow_year(year=2022):
  
    # Get the URL
    URL = "https://www.footywire.com/afl/footy/brownlow_medal?year="+str(year)+"&s=V"
    soup = make_soup(URL)
    
    # Extract out the table. we search for the table that's exactly 688
    brownlow_table_html = [a for a in soup.find_all('table') if a.get('width')=='688'][0]
    flat_table_data = [a for a in brownlow_table_html.find_all('td')]
    header_elements = flat_table_data[0:9]
    body_elements = flat_table_data[9:]
    headers = [clean_text(a.text) for a in header_elements]
    data_list = []
    for e in range(0,len(body_elements),len(headers)):
        row_data = body_elements[e:e+len(headers)]
        data_list.append(parse_row_data_bl(row_data,headers))
    return pd.DataFrame(data_list)

brownlow_data = get_brownlow_year()
brownlow_data

,Player,Team,V,3V,2V,1V,Played,Polled,V/G,player_id,team_id
0,Patrick Cripps W,Blues,29,7,2,4,21,13,1.38,carlton-blues--patrick-cripps,carlton-blues
1,Lachie Neale,Lions,28,7,3,1,22,11,1.27,brisbane-lions--lachie-neale,brisbane-lions
2,Touk Miller,Suns,27,4,6,3,22,13,1.23,gold-coast-suns--touk-miller,gold-coast-suns
3,Andrew Brayshaw,Dockers,25,5,4,2,22,11,1.14,fremantle-dockers--andrew-brayshaw,fremantle-dockers
4,Clayton Oliver,Demons,25,4,5,3,21,12,1.19,melbourne-demons--clayton-oliver,melbourne-demons
...,...,...,...,...,...,...,...,...,...,...,...
202,Tim Membrey,Saints,1,0,0,1,22,1,0.05,st-kilda-saints--tim-membrey,st-kilda-saints
203,Sebastian Ross,Saints,1,0,0,1,21,1,0.05,st-kilda-saints--sebastian-ross,st-kilda-saints
204,Jake Lloyd,Swans,1,0,0,1,21,1,0.05,sydney-swans--jake-lloyd,sydney-swans
205,Willie Rioli,Eagles,1,0,0,1,13,1,0.08,west-coast-eagles--willie-rioli,west-coast-eagles


In [22]:

def parse_row_data_con(row_data,headers,team):
    data_dict = {headers[i]:clean_text(row_data[i].text) for i in range(len(row_data))}
    pid = get_pids(row_data)
    data_dict['player_id'] = clean_text(pid)
    data_dict['team_id'] = clean_text(team)
    return data_dict
    
    
def get_table_team(team):
    
    print(f'Scraping contracts for {team}')
    
    # Get the URL
    URL = "https://www.footywire.com/afl/footy/to-"+team
    soup = make_soup(URL)
    
    # Extract out the table. we search for the table that's exactly 688
    contract_table_html = [a for a in soup.find_all('table') if a.get('width')=='688'][0]
    
    # For each element, we extract the text and replace the 'xa0' (non-breaking character) with ''
    flat_table_data = [a for a in contract_table_html.find_all('td')]

    header_elements = flat_table_data[0:4]
    body_elements = flat_table_data[4:]
    headers = [clean_text(a.text) for a in header_elements]
    data_list = []
    for e in range(0,len(body_elements),len(headers)):
        row_data = body_elements[e:e+len(headers)]
        data_list.append(parse_row_data_con(row_data,headers,team))
    
    return pd.DataFrame(data_list)

TEAMS = [
"adelaide-crows",
"brisbane-lions",
"carlton-blues",
"collingwood-magpies",
"essendon-bombers",
"fremantle-dockers",
"gold-coast-suns",
"greater-western-sydney-giants",
"hawthorn-hawks",
"kangaroos",
"st-kilda-saints",
"sydney-swans",
"west-coast-eagles",
"western-bulldogs",
"port-adelaide-power",
"melbourne-demons",
"geelong-cats",
"richmond-tigers"]

contract_data = pd.concat([get_table_team(team) for team in TEAMS[0:3]], axis =0)    


Scraping contracts for adelaide-crows
Scraping contracts for brisbane-lions
Scraping contracts for carlton-blues


In [24]:
contract_data

,Name,Final Year,Years Service*,Status,player_id,team_id
0,Andrew McPherson,2023,6,Non-Free Agent,adelaide-crows--andrew-mcpherson,adelaide-crows
1,Ben Keays,2024,5,Non-Free Agent,adelaide-crows--ben-keays,adelaide-crows
2,Billy Dowling,2024,2,Non-Free Agent,adelaide-crows--billy-dowling,adelaide-crows
3,Brayden Cook,2024,4,Non-Free Agent,adelaide-crows--brayden-cook,adelaide-crows
4,Brodie Smith,2024,14,Unrestricted Free Agent,adelaide-crows--brodie-smith,adelaide-crows
...,...,...,...,...,...,...
40,Sam Philp,2023,4,Non-Free Agent,carlton-blues--sam-philp,carlton-blues
41,Sam Walsh,2026,8,Restricted Free Agent,carlton-blues--sam-walsh,carlton-blues
42,Tom De Koning,2023,6,Non-Free Agent,carlton-blues--tom-de-koning,carlton-blues
43,Zac Fisher,2025,9,Restricted Free Agent,carlton-blues--zac-fisher,carlton-blues


In [25]:
brownlow_data.head()

,Player,Team,V,3V,2V,1V,Played,Polled,V/G,player_id,team_id
0,Patrick Cripps W,Blues,29,7,2,4,21,13,1.38,carlton-blues--patrick-cripps,carlton-blues
1,Lachie Neale,Lions,28,7,3,1,22,11,1.27,brisbane-lions--lachie-neale,brisbane-lions
2,Touk Miller,Suns,27,4,6,3,22,13,1.23,gold-coast-suns--touk-miller,gold-coast-suns
3,Andrew Brayshaw,Dockers,25,5,4,2,22,11,1.14,fremantle-dockers--andrew-brayshaw,fremantle-dockers
4,Clayton Oliver,Demons,25,4,5,3,21,12,1.19,melbourne-demons--clayton-oliver,melbourne-demons


In [28]:
pd.merge(contract_data, 
         brownlow_data, 
         left_on='player_id', 
         right_on='player_id',
         how = 'left',
         suffixes=('_contract', '_brownlow')).to_csv('brownlow.csv')